# In the name of Allah

### Please input data addrs:

In [ ]:
# list of data which should be normalized:
bijankhan_zip_addr = "'Bijankhan.rar'"

persianHealth_addr = "'persian_health.zip'"

utdParags_addr = "UptoDate_parags.zip"

wikiEn_addr = "en_papers.json"
wikiFa_addr = "fa_papers.json"

### Unzip data

In [ ]:
# first level data is zipped or rarred so we ask to unzip
!unrar x '$bijankhan_zip_addr'
!unzip -q $persianHealth_addr
!unzip -q $bijankhan_data_addr
!unzip -q $utdParags_addr
!rm /content/health/README.md
!rm -r /content/health/.ipynb_checkpoints

### Install required package

In [ ]:
!pip -q install hazm

     |████████████████████████████████| 316 kB 13.9 MB/s 
     |████████████████████████████████| 1.4 MB 57.8 MB/s 
     |████████████████████████████████| 233 kB 64.9 MB/s 


### Import required packages

In [ ]:
import re
import pandas as pd
import nltk
import pickle
import tqdm
import json 
import pandas as pd
import numpy as np
import os
from hazm import Normalizer, SentenceTokenizer
from nltk.tokenize import sent_tokenize
from itertools import chain
#it can be use if we want punctuations but not all the time, in this code we used ready one
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Define normalization functions:

In [ ]:
#partially based on https://stackoverflow.com/questions/41174566/how-to-normalize-persian-texts-with-hazm

def en_remove_punc(s):
    #list of punctuations, more complete than ready lists
    punc = '"#\'*+,-/:;<=>@[\]^_`{|}~\'●,•()»«–‑-،؛−٫—' # "●,•" not happen in common texts but in our data we saw them sometimes
    table = str.maketrans(dict.fromkeys(punc, ' ')) 
    new_s = s.translate(table) 
    new_s = ' '.join(new_s.split())
    return new_s


def fa_remove_punc(s):
    punc = '"#\'*+,-:;<=>@[\]^_`{|}~\'●,•()»«–‑-،؛−—'
    table = str.maketrans(dict.fromkeys(punc, ' ')) 
    new_s = s.translate(table) 
    new_s = ' '.join(new_s.split())
    return new_s

def en_normalizer(text):
    # using lower characters is common in different english tasks
    text = text.lower()
    # remove \xa0 which is useless in our project
    text = text.replace('\xa0','')
    #text = text.replace('-',' ')
    # removeing citations from text which is not usable here
    text = re.sub(r"\[[\d| ]+\]", " ", text)
    text = en_remove_punc(text)
    #text = re.sub(r"(.)\.([^0-9]|\n|$)", r"\1 . \2", text)
    text = re.sub(r"(\w{2,}| )\.([^0-9]|\n|$)", r"\1 . \2", text)
    # here convert commom puctuations into based format, as example latin to english, or in persian from english to persian
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"؟", " ؟ ", text)
    text = re.sub(r" +", " ", text)
    return text


def fa_normalizer(text):
    text = arToPersianChar(text)
    text = arToPersianNumb(text)
    text = text.replace('\xa0','')
    #text = text.replace('-',' ')
    text = text.replace('ٔ', '')
    text = fa_remove_punc(text)
    # more_normalization_function()
    normalizer = Normalizer(persian_style = False, punctuation_spacing = False, affix_spacing = False)
    text = normalizer.normalize(text)
    # this two next line are usable in persian texts, but it is common to see different behavioural from writers, sometimes using them and sometimes no.
    text = text.replace('\u200c',' ')
    text = text.replace('\u200b',' ')
    #here we manage '.'s in end of sentences, but it can understand to avoid float numbers dots. 
    text = re.sub(r"(\w{2,}| )\.([^0-9]|\n|$)", r"\1 . \2", text)
    # still same as above comments, convert signs to thier normal forms
    text = re.sub(r'([\d+])\.([\d+])', r'\1٫\2', text)
    text = re.sub(r'([\d+])/([\d+])', r'\1٫\2', text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r" +", " ", text)
    text = re.sub(r" +", " ", text)
    return text


def arToPersianNumb(number):
    dic = {
        '١': '۱',
        '٢': '۲',
        '٣': '۳',
        '٤': '۴',
        '٥': '۵',
        '٦': '۶',
        '٧': '۷',
        '٨': '۸',
        '٩': '۹',
        '٠': '۰',
    }
    return multiple_replace(dic, number)


def arToPersianChar(userInput):
    dic = {
        'ك': 'ک',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
    }
    return multiple_replace(dic, userInput)

def multiple_replace(dic, text):
    # it help us to have less line of codes because it do different replaces in one line
    pattern = "|".join(map(re.escape, dic.keys()))
    return re.sub(pattern, lambda m: dic[m.group()], str(text))

### Normalize English and Farsi wikipedia data

In [ ]:
with open(wikiFa_addr) as f:
  wikiFa_data = json.load(f)

In [ ]:
with open(wikiEn_addr) as f:
  wikiEn_data = json.load(f)

In [ ]:
wiki_en_df = pd.DataFrame(wikiEn_data.items(), columns=['Key', 'Value'])

In [ ]:
wiki_en_parags_normd = []
deleted = []
for _, doc in tqdm.tqdm(wiki_en_df.iterrows()):
  # each '\n' means new paragraph, so it is a good splitting criteria!
  for parag in doc['Value'].split('\n'):
    parag_normd = en_normalizer(parag)
    if parag_normd == '':
      deleted.append(parag)
      continue
    wiki_en_parags_normd.append(parag_normd)

27019it [01:46, 253.58it/s]


In [ ]:
wiki_fa_df = pd.DataFrame(wikiFa_data.items(), columns=['Key', 'Value'])

In [ ]:
wiki_fa_parags_normd = []
deleted_fa = []
for _, doc in tqdm.tqdm(wiki_fa_df.iterrows()):
  for parag in doc['Value'].split('\n'):
    parag_normd = fa_normalizer(parag)
    if parag_normd == '':
      deleted_fa.append(parag)
      continue
    wiki_fa_parags_normd.append(parag_normd)

16506it [00:59, 278.50it/s]


#### Save wikipedia normalized data

In [ ]:
with open('./wiki-en_parags_normd.txt', 'w', encoding = 'UTF-8') as f:
  for text in wiki_en_parags_normd[:-1]:
    f.write(text+'\n')
  else:
    f.write(wiki_en_parags_normd[-1])

In [ ]:
with open('./wiki-fa_parags_normd.txt', 'w', encoding = 'UTF-8') as f:
  for text in wiki_fa_parags_normd[:-1]:
    f.write(text+'\n')
  else:
    f.write(wiki_fa_parags_normd[-1])

#### Normalize English and Farsi wikipedia titles

In [ ]:
with open("./orphans_merged.json") as f:
  orphans_merged = json.load(f)
om = []
for i in orphans_merged:
    om.append(en_normalizer(i))

In [ ]:
with open('./orphans_merged_normd.txt', 'w') as f:
    for item in om:
        f.write("%s\n" % item)

In [ ]:
with open("./pairs_merged.json") as f:
  pairs_merged = json.load(f)
pm = []
for i in pairs_merged:
    pm.append([en_normalizer(i[0]),fa_normalizer(i[1])])


In [ ]:
with open('./pairs_merged_normd.txt', 'w') as f:
    for item in pm:
        f.write("%s\n" % item)

### Normalize Farsi Bijankhan data

In [ ]:
with open(bijankhan_processed_addr, 'r') as f:
  bjk = f.read()

#### Get text from POS tagged bijankhan data. Then get paragraphs of text which are seperated by \# sign 

In [ ]:
bjk_splt = bjk.split('\n')
bjk_splt_splt = [row.split('  ')[0] for row in bjk_splt]
bjk_text = ' '.join(bjk_splt_splt)
bjk_text_splt = bjk_text.split("#")

In [ ]:
bjk_text_splt_normd = []
for doc in bjk_text_splt:
  bjk_text_splt_normd.append(fa_normalizer(doc))

#### Save normalized data

In [ ]:
with open('./bijankhan_parags_normd.txt', 'w', encoding = 'UTF-8') as f:
  for text in bjk_text_splt_normd[:-1]:
    f.write(text+'\n')
  else:
    f.write(bjk_text_splt_normd[-1])

### Normalize Farsi health data from Namnak and HiDoctor websites (from course github exploring datasets)

In [ ]:
fa_health_jsons = [os.path.join('health',x) for x in os.listdir('health')]

In [ ]:
with open(fa_health_jsons[0], 'r') as f:
  text = json.load(f)

In [ ]:
fa_health_parags_normd = []
for js in fa_health_jsons:
  with open(js, 'r') as f:
    text = json.load(f)
    for doc in text:
      for parag in doc['paragraphs']:
        fa_health_parags_normd.append(fa_normalizer(parag))

#### Save normalized data

In [ ]:
with open('./persian-health_parags_normd.txt', 'w', encoding = 'UTF-8') as f:
  for text in fa_health_parags_normd[:-1]:
    f.write(text+'\n')
  else:
    f.write(fa_health_parags_normd[-1])

### Normalize UptoDate English papers

In [ ]:
utd_parags_df = pd.read_csv(utdParags_addr)

In [ ]:
utd_parags_normd = list(utd_parags_df['text'].apply(en_normalizer))

#### Save normalized data

In [ ]:
with open('./uptodate_parags_normd.txt', 'w', encoding = 'UTF-8') as f:
  for text in utd_parags_normd[:-1]:
    f.write(text+'\n')
  else:
    f.write(utd_parags_normd[-1])

### Write all English and Farsi data in seperate files

In [ ]:
all_en_parags = [wiki_en_parags_normd, utd_parags_normd]
all_fa_parags = [bjk_text_splt_normd, fa_health_parags_normd, wiki_fa_parags_normd]

In [ ]:
with open('./all-en_parags_normd.txt', 'w', encoding = 'UTF-8') as f:
  for dateset in all_en_parags:
    for text in dateset:
      f.write(text+'\n')

In [ ]:
with open('./all-fa_parags_normd.txt', 'w', encoding = 'UTF-8') as f:
  for dateset in all_fa_parags:
    for text in dateset:
      f.write(text+'\n')